### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

sys.path.append("../../data_analysis/")
import helper as hp

In [3]:
savepath = "C04_full_new_round/"
os.makedirs(savepath, exist_ok=True)

In [4]:
all_seqs = []

In [5]:
df_esm1v = pd.read_csv("C01_elisa_extract_top_seq/esm1v.csv")
esm1v = df_esm1v["AA"].to_list()
all_seqs.extend(esm1v)
len(all_seqs)

46

In [6]:
df_cterm = pd.read_csv("C02_pats_peptides/cterm_pat.csv")
cterm = df_cterm["AA"].to_list()
all_seqs.extend(cterm)
len(all_seqs)

53

In [7]:
df_nterm = pd.read_csv("C02_pats_peptides_nterm/nterm_pat.csv")
nterm = df_nterm["AA"].to_list()
all_seqs.extend(nterm)
len(all_seqs)

60

In [8]:
df_patent = pd.read_csv("C03_new_patent_extraction/new_patent.csv")
patent = df_patent["AA"].to_list()
all_seqs.extend(patent)
len(all_seqs)

90

In [9]:
len(set(all_seqs))

90

In [10]:
all_ids = [f"seq{i}" for i in range(len(all_seqs))]
assert len(all_ids) == len(set(all_seqs))

df_to_opti = pd.DataFrame({'ID': all_ids, 'Peptide': all_seqs})
df_to_opti.head(2)

,ID,Peptide
0,seq0,CCQCCCLCCAFCCCCKCELCPK
1,seq1,ACQVQHCCCACCCYCCVCCFCC


In [11]:
df_to_opti.to_csv(f'{savepath}df_to_opti.csv', sep='\t', index=False)

IDT

In [12]:
d_idt = {
    1: "TGTTGCCAGTGTTGTTGTCTGTGCTGTGCCTTCTGCTGCTGTTGCAAATGCGAATTATGTCCTAAA",
    2: "GCTTGCCAGGTGCAACATTGCTGCTGTGCCTGTTGTTGCTACTGTTGCGTGTGTTGCTTCTGTTGC",
    3: "GCGTGCCAAGTACAACACTGTTGCTGTTGCTGTTGCTGTTATTGCTGTGTCTGCTGCTTCTGCTGT",
    4: "CAGGGTTGTGTGTGTTGCCTGTGTTGTTGCTGCTGCTGCTGTAAATGCGTCGAATGTTTTTGCAAG",
    5: "TGCTGCCAGTGCTGTTGTCTGTGCTGCTGTTTCTGTTGTTGTTGCTGCTGTGAACTTTTTCCGAAA",
    6: "TGCGGCTGTGTTTGCCATTTGTGCTGTTGTTGTTGTTGCTGCAAATGCGTGGAATGCTTTTGCAAA",
    7: "TGTGGGTGCGTGTGTCATTGTTGCTGCTGTTGCTGCTGTTGCAAGTGCGTTGAATGTTTTTGCAAG",
    8: "TGCTGCCAATGCTGCTGCTTGTGTTGCGCCTTTTGTTGTTGCTGTTGTTGCTGCTTGTTCCCAAAG",
    9: "GCGTGTCAGGTCCAACACTGCTGTTGTTGCTTTTGCTGTTATTGTTGCTGCTGCTGCTTCTGCTGC",
    10: "CAGGGCTGTGTATGCTGCTTATGTTGTTGTTGCTGCTGCTGCTGTTGTGTGGAATGTTTTTGCAAG",
    11: "GCTTGTCAAGTCCAACATTGCTGCTGTTGCTTCTGCTGTTGTTGCTGTGTATGCTGTTTCTGCTGC",
    12: "TGTTGCTGTTGTTGCTGCTTATGTTGCTGCTTTTGTTGCTGTTGCAAGTGCGAGTTGTTTCCTAAG",
    13: "GCTTGTCAGGTTCAGCATTGCTGTTGTGCATTTTGTTGCTATTGTTGTTGTTGTTGTTTTTGCTGC",
    14: "CAAGGCTGCGTCTGCCATTTATGCTGTTGTTGTTGTTGTTGTTGCTGTGTGGAATGTTTCTGTAAG",
    15: "CAAGGCTGCGTATGCTGTTGCTGCTGCTGCTGTTGTTGCTGCAAGTGTGTAGAGTGCTTTTGTAAA",
    16: "TGTGGATGCGTTTGCCATTTATGCTGCTGCTGCTGTTGTTGCAAATGCGTTTGTTGCTTTTGCAAA",
    17: "TGCTGCCAGGTGCAACATTGCTGCTGTGCTTGTTGTTGCTACTGCTGCGTATGTTGTTTCTGTTGT",
    18: "TGTTGTTGTTGTTGCTGTTTATGCTGTGCCTTCTGTTGTTGTTGCAAATGTGAATTATTCCCAAAG",
    19: "CAGGGATGCGTGTGCTGCCTGTGCTGCTGCTGCTGTTGTTGCAAGTGTGTTTGCTGCTTCTGCAAG",
    20: "TGCTGTTGCTGCTGTTGCCTGTGCTGTGCTTTCTGTTGTTGCTGCTGTTGCGAACTGTTCCCTAAA",
    21: "TGTTGCCAATGTTGCTGTCTGTGCTGTTGCTTCTGTTGCTGCTGCAAGTGTGAGTTATGTCCTAAG",
    22: "GCCTGCCAGGTTCAGCACTGTTGCTGCGCCTGCTGTTGCTATTGTTGCTGCTGCTGTTTTTGCTGC",
    23: "GCATGTCAAGTTCAGCACTGTTGCTGTGCGTTCTGTTGCTGTTGTTGCTGCTGCTGTTTTTGTTGT",
    24: "GCATGTCAAGTTCAGCACTGTTGCTGTGCATGTTGCTGTTGCTGCTGTGTTTGCTGCTTTTGCTGT",
    25: "CAAGGGTGTGTTTGTCATTTATGCTGCTGTTGTTGCTGCTGTAAATGCGTCTGTTGCTTTTGTAAG",
    26: "TGTTGCCAGTGCTGTTGTCTGTGCTGTGCCTTCTGTTGCTGTTGTAAATGCTGTTTATTTCCGAAG",
    27: "TGTGGGTGTGTATGCTGCTTGTGCTGTTGTTGTTGCTGTTGCAAATGTGTTGAATGTTTCTGCAAG",
    28: "TGCGGACAAGTTTGTTGTTGTCAGGCATGTTTTTCTTGCTGCTGCTGTTGCTGCTTGTTCTGTAAA",
    29: "TGCTGCCAATGTTGTTGCTTATGCTGTGCCTTCTGCTGCTGTTGCTGTTGTGAGCTTTTCCCCAAG",
    30: "CAGGGTTGTGTATGCCATTGCTGTTGTTGCTGCTGTTGCTGTTGTTGCGTGGAATGCTTCTGTAAG",
    31: "TGTTGCCAGGTGCAGCACTGCTGCTGTGCGTTTTGTTGCTGCTGTTGCGTGTGTTGCTTTTGTTGT",
    32: "TGTGGATGCGTTTGTCATTTGTGCTGCTGTTGCTGTTGCTGCTGCTGTGTAGAGTGTTTTTGTAAA",
    33: "TGCTGTCAGTGTTGTTGCTTGTGCTGCTGTTTCTGCTGTTGCTGTAAATGCGAGCTGTTTCCTAAG",
    34: "CAGGGTTGCGTCTGTCACTGCTGCTGCTGTTGCTGCTGCTGTAAGTGCGTATGCTGTTTTTGCAAG",
    35: "TGCTGCCAGTGTTGTTGCCTGTGTTGTGCTTTTTGCTGTTGCTGCAAGTGCTGCCTGTGTCCTAAG",
    36: "CAAGGATGCGTGTGTCACCTGTGCTGTTGCTGCTGTTGCTGTTGCTGCGTCTGCTGCTTTTGCAAG",
    37: "GCCTGTCAAGTCCAGCATTGCTGTTGCTGCTTCTGCTGTTACTGCTGCGTCTGCTGCTTCTGTTGT",
    38: "TGTTGCCAGGTGCAGCATTGCTGTTGCGCGTTTTGTTGTTATTGTTGTTGTTGTTGCTTCTGCTGT",
    39: "TGTTGCCAGGTACAACATTGTTGTTGTTGTTTCTGCTGTTACTGCTGTGTATGCTGTTTTTGCTGT",
    40: "TGTTGCCAATGCTGCTGTCTGTGTTGCGCCTTTTGTTGTTGTTGTTGTTGTGAACTGTGCCCCAAG",
    41: "GCTTGCCAGGTCCAACATTGTTGCTGTGCCTTCTGCTGTTGTTGTTGCGTTTGTTGCTTTTGCTGT",
    42: "TGTTGTCAGGTCCAGCATTGCTGTTGCGCCTTTTGCTGTTACTGTTGTGTTTGTTGTTTTTGTTGC",
    43: "TGTTGTTGTTGTTGCTGTCTTTGTTGTGCGTTTTGTTGTTGCTGTAAGTGCTGTTTGTTTCCAAAA",
    44: "CAAGGTTGTGTCTGTCACTGTTGTTGCTGTTGTTGTTGCTGCAAATGCGTAGAATGTTTCTGTAAA",
    45: "TGCTGTTGCTGCTGCTGCCTGTGCTGCGCTTTTTGTTGTTGTTGCAAGTGCGAATTATGCCCCAAG",
    46: "TGTTGTCAATGTTGCTGCCTGTGTTGCTGCTTTTGCTGCTGTTGTAAATGCTGTCTGTTTCCAAAA",
    47: "AGTTTGGAAGAGTTGTTCGAGCGTAATCGCCGCTTAAAGGAGGAATTA",
    48: "AGTGAGTTTGTAGAGATGTTTCAGCGCATCCGCGAACTTCGTGAA",
    49: "TCTTTATCAGAGCTTTTTGAGCGTAATCGTGAATTAAAGGAACGCCTT",
    50: "TCTGATTTAGCCGCACTGTTTCAGCGTGTCCAAGAGCTTAAAGAA",
    51: "GGAGAAATCGTGGAACTTTACAAACGCGTACAAGAACTGAAAGAGAAATTA",
    52: "TCGCTGGTCAGCTTATATGAACGTGTGCAGCGTTTAAAGGAAGAGTTG",
    53: "AGTCTGGTGGAATTGTTCAAACGCGTGCAAGAGCTTAAAGAGAAA",
    54: "CACATGATGGATGAGATTAAGAAACTTGCTAACGAGTCGATTAAGGAGTTTGAGGAATACGCGGAAAAAGTACTGGAAGAATTAAAAGAG",
    55: "GATGAAATGAATCGCCTTGCAAATGAAGCAATCGCAGAGATGGAGGCATACGCAGCTGCTCAGCTTGAAGCGCTGAAAGCACGTGTTGCA",
    56: "CATATGATGGACGAAATTATCGCTAAGGCCAATGAGGCAATCGAGGAATTCGAGGCTTACGCGCGCGCACAGTTAGCAGAATTGAAAGCT",
    57: "GATGACATGAATCGCAAGGTAAATGAGGCTATCGCTGAGATGGAGGAGTATGCAGAAGAAGTCCTTAAAGAGTTGAAAGAGGAAGTAGAG",
    58: "AGTGAGATGAATGAGTTAGCGAATGAATCCATTGCGGAAATGGAAGCGTACGCACGCCGCGTTAATGAAGAACTT",
    59: "TCATCCCTTATGGAGGAGATTCGTGCCTTAGCCAAAGCAGAAGTAGCCGAATTTGAGGCAGAAGTTGAGAAACGCCTGGCGGAGATGAAG",
    60: "TCATCGATGATGGAAAATGCGATTGCTTTGGCTAACGAAGAGATTGCGCGCTTTGAGGCGTACGCTGAGCGCGTCGTTGCGGAGTTAAAG",
    61: "CAGGGACAGGTTCAACACTTGCAAGCGTGCTTCTCCGCCTATAAGAAGGTGGAACTGTTTCCGAAG",
    62: "CAAGGTCAGTGCTGTTGTTTGCAATGCGCTTGCTGTCAATACTGCAAGTGTGAGTGCTTTCCTTGC",
    63: "TGTGCTTGTTGCCAAGCTTGTTGCGCTTGTTTCAGTTGCGCAAAATGCGCTGAGTGTTTTTGCAAG",
    64: "TGTGGGCAAGTATGTCACCTTCAGTGTGCATTTTCGGGCTATAAGAAGTGCGAATTATTTCCCAAATGT",
    65: "TGTGGGCAGTGCTGTCATTTACAATGTTGTTTCTGTCAATACAAAAAATGTGAGCTGTTTCCAAAGTGT",
    66: "TGCTGTTGCGTTTGCTGTTGCCAAGCTGCTTGCAGCGCGTATTGCTGCTGCTGTGCCTGTCCAAAG",
    67: "CAGTGTTGCGTGTGTTGCGCACAAGCATGCTTCTGTCAATATAAAAAATGCGAATGTTGCTGTGCA",
    68: "TGCGGGTGTGTGCAGCACTGCCAAGCAGCTTTCTGTCAGTACGCAAAAGTTTGCCTGTTTGCTAAA",
    69: "CAGGGGTGCTGCTGTCACTGCCAATGTGCGTTTTGTCAATACTGTGCCTGTGAGCTGTGCCCCAAG",
    70: "CAATGTCAGGTTCAGCACTTACAGGCTGCGTTCTGCCAGTGCAAGTGCTGTGAGCTGTGCCCCAAA",
    71: "CAGGGACAGTGTCAACATTGTCAAGCCGCCTTTTCTCAGGCATGTAAAGCGGAGTTGTTTCCAAAG",
    72: "CAGGGATGTTGCCAACATTTATGTGCAGCGTTCAGCCAGTATAAGTGCGTCGAATTATTCCCCAAA",
    73: "CAAGGGCAGGTTCAGCACTGTTGTGCTGCGTTTTCACAGTACAAGAAGGTTGAGCTGTTTCCAAAG",
    74: "CAGGGTTGTGTCGCGTGCTTGCAATGTGCGTTTTCGCAGTACAAAAAATGTGCTGCGTGCCCTTGT",
    75: "CAGGCTTGCGCCCAATGCCTGTGCGCCGCTTTTTCACAGTATGCTAAGGTGGAGCTGTTCCCTAAA",
    76: "CAGGCACAAGTACAACACTTACAAGCGGCGTTCAGTCAATGCTGCTGTGTTGAATTGTTTTGCAAA",
    77: "TGCTGTCAGTGCTGTCACTGCCAGTGTTGCTTTTCTCAATACTGCAAAGTGTGCTGTTTTCCAAAA",
    78: "CAAGGCCAAGTACAACACTTATGTGCGTGCTGCTCTTGTTACAAGAAGGTCTGTTGTTGCTGCTGC",
    79: "TGCGGTCAAGTTTGCCACTTACAGTGCGCATTCAGCCAATACAAAAAATGCGAGTTATTTCCCAAATGT",
    80: "TGCGGCGCAGTTCAGTGTTGCTGTGCGTGCTTCTGCTGCTGTGCCAAAGTAGAATGCTTTTGTGCT",
    81: "CAGTGTCAGTGTCAGTGCCTTCAGGCAGCTTTTTGTTGTTATTGCAAGGTCGAGTTGTGTCCAAAG",
    82: "CAAGCCTGCGTACAACACTGCCAATGTTGCTTCTGCCAGGCCAAAGCGGTAGAGCTTTGTCCGAAG",
    83: "TGCCAGGGACAAGTGCAGCACTTACAAGCCGCATTCAGCCAGTACAAAAAGTGCGAATTGTTTCCTAAGTGC",
    84: "TGCTGTCAAGTCCAGTGTTGTTGTGCCGCGTGCTGTCAGTATTGCAAGGTCGAATGTTTCCCTAAA",
    85: "CAAGCGTGTGTCCAAGCATTGGCAGCATGTTTCAGTCAATGCAAGTGCGTCTGTCTGTGTCCATGC",
    86: "TGTGGTTGCTGTTGCCACCTGTGTTGCTGTTTCTGTTGCTACAAAAAGTGCGAATTATTCCCTAAATGC",
    87: "CAGGGTCAGGTCCAATGCCTGTGTTGTTGCTGCTCTCAATACAAAAAGGTAGAATGCTTTCCCAAG",
    88: "CAAGGTCAGGTACAGTGTTGCCAAGCGGCCTGCTCCCAGTATAAGAAAGTTGAATTGTTTCCCTGT",
    89: "CAAGGATGCGTCCAGTGTCTGCAATGTGCCTGTTGTGCCTGCAAATGTTGCTGCTTGTTCTGTGCT",
    90: "TGCGGATGTTGTGCTGCGCTGTGCGCCTGTTGCTCCTGTTATAAGAAAGCTTGCTGCTTTCCAGCA",
}

len(d_idt)

90

In [13]:
from Bio.Seq import Seq

aa_seq_check = []
for key, value in d_idt.items():
    dna_sequence = Seq(value)
    amino_acid_sequence = dna_sequence.translate()
    aa_seq_check.append(str(amino_acid_sequence))

len(aa_seq_check)


90

In [14]:
ok=0
for seq in aa_seq_check:
    if seq  in all_seqs:
        ok+=1
ok

90

In [15]:
assert ok==len(aa_seq_check)

In [16]:
front = "GCGAATTAATACGACTCACTATAGGGAGACCACAACGGTTTCCCTCTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGAAAATCGAAGAAGATTACAAGGATCACGATGGAGACTATAAGGATCACGACATCGATTACAAAGACGATGACGACAAATCAGGCCATCACCATCATCATCATCATCACTCGTCAGGC"
end = "TAGTAGTAAGGAGGATCCCGGGAATTCTCGAGTAAGGTTAACCTGCTAGCATAACCCCTTGGGGCCTCTAAACGGGTCTTGAGGGGTTTTTTG"
len(front), len(end)

(206, 93)

In [17]:
d_final_seqs = {}
for key, value in d_idt.items():
    dna_sequence = front + value + end
    d_final_seqs[key] = dna_sequence
len(d_final_seqs)

90

In [18]:
df_final_seqs = pd.DataFrame.from_dict(d_final_seqs, orient='index', columns=['Sequence'])
df_final_seqs.head(2)

,Sequence
1,GCGAATTAATACGACTCACTATAGGGAGACCACAACGGTTTCCCTC...
2,GCGAATTAATACGACTCACTATAGGGAGACCACAACGGTTTCCCTC...


In [19]:
len(d_final_seqs)

90

In [20]:
df_final_seqs.to_csv(f'{savepath}ml_patent.csv', index_label='ID')